# Create meeting minutes from an Audio file

In [1]:
# imports

import os
import requests
import ollama
from IPython.display import Markdown, display, update_display
from openai import OpenAI

from dotenv import load_dotenv
from openai import OpenAI
import warnings
from huggingface_hub import configure_http_backend

In [2]:
# Constants

AUDIO_MODEL = "whisper-1"

In [3]:
# mp3 path
audio_filename = "./resources/audio_sample.mp3"

In [4]:
# set the session unverified
def backend_factory() -> requests.Session:
    session = requests.Session()
    session.verify = False
    return session

warnings.filterwarnings("ignore")
configure_http_backend(backend_factory=backend_factory)

In [5]:
# Sign in to OpenAI using Secrets in Colab
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [6]:
import requests

api_url = "https://api.openai.com/v1/audio/transcriptions"

# Open the audio file
with open(audio_filename, "rb") as audio_file:
    # Prepare the data for the request
    files = {"file": audio_file}
    data = {"model": AUDIO_MODEL, "response_format": "text"}
    headers = {"Authorization": f"Bearer {openai_api_key}"}

    # Make the request without SSL verification
    response = requests.post(api_url, files=files, data=data, headers=headers, verify=False)

transcription = response.text
# Print the transcription
print(transcription)

brings you in here today? Yeah, I have this pain in my chest. Okay, and where is the pain exactly? It's just right over on the on the left side. Okay, and when did this pain start? It started just 30 minutes ago. Okay, and did it just come on randomly or were you doing something strenuous? I was just shuttling the driveway and it came on. Okay, and has that pain been getting worse at all over the last one and a half an hour? No, it just came on suddenly and it's sorry, yeah, the pain has been there this whole time and it's gotten worse ever since it started. Okay, and how would you describe the pain? Is it kind of like an aching pain or is it a sharp or tightness kind of pain? How would you describe it? It feels dull. I feel like there's a lot of pressure on my chest. Okay, and how would you rate the pain right now on a scale of zero to ten? Zero being the least amount of pain you've felt in your life, ten being the worst. Seven. Seven, okay. Have you had any similar episodes before? N

In [7]:
# make LLAMA 3.2 summarize the transcription

system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, in markdown."
user_prompt = f"Below is an extract transcript from a conversation. Please write minutes in markdown, including a summary with any relevant discussion points;\n{transcription}"

response = ollama.chat(
                model="llama3.2",
                messages=[
                    {
                        "role": "system",
                        "content": system_message
                    },
                    {
                        "role": "user",
                        "content": user_prompt,
                    }
                ],
            )
llm_response = response["message"]["content"]
display(Markdown(llm_response))

**Meeting Minutes: Chest Pain Discussion**
=====================================

**Summary**
-----------

A patient presented with sudden onset chest pain that has been worsening over 30 minutes. The pain is described as dull and feels like there's pressure on the left side of the chest. The patient has a history of high cholesterol, high blood pressure, and smokes cigarettes, but does not have any previous symptoms or diagnoses related to heart problems.

**Key Discussion Points**
------------------------

*   Sudden onset chest pain that worsened over 30 minutes
*   Pain is described as dull and feels like pressure on the left side of the chest
*   Patient has a history of high cholesterol, high blood pressure, and smokes cigarettes
*   No previous symptoms or diagnoses related to heart problems
*   Patient reports feeling short of breath since yesterday when the pain started

**Takeaways**
-------------

*   The patient's sudden onset chest pain warrants further evaluation and potential emergency medical attention.
*   The patient's history of high cholesterol, high blood pressure, and smoking increases their risk for cardiovascular disease.
*   The patient's symptoms of feeling short of breath since yesterday when the pain started should be monitored closely.

**Action Items**
----------------

*   Schedule an immediate appointment with a cardiologist or primary care physician to evaluate the patient's chest pain
*   Order a 12-lead electrocardiogram (ECG) and other diagnostic tests as needed
*   Advise the patient to report any worsening symptoms or changes in their condition immediately